
## [ Part-2 ]  Training and Classification

In [1]:
# Importing libraries
import os
from __future__ import print_function
# from keras.models import Sequential
# from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [2]:
testDf = pd.read_csv('test_indessa.csv')

In [23]:
X = np.load("x_train.npy")
Y = np.load("y_train.npy")
test = np.load("test.npy")
print ("Training data: ",X.shape)
print ("Testing data: ",test.shape)
print (X[0])
print (test[0])

Training data:  (532428L, 29L)
Testing data:  (354951L, 29L)
[  1.43500000e+04   1.43500000e+04   1.43500000e+04   3.60000000e+01
   1.91900000e+01   4.00000000e+00   2.20000000e+01   9.00000000e+00
   4.00000000e+00   2.87000000e+04   1.00000000e+00   2.00000000e+00
   9.00000000e+00   3.38800000e+01   0.00000000e+00   1.00000000e+00
   1.40000000e+01   1.00000000e+00   2.25150000e+04   7.31000000e+01
   2.80000000e+01   0.00000000e+00   1.17384000e+03   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   2.60000000e+01
   0.00000000e+00]
[  1.40000000e+04   1.40000000e+04   1.40000000e+04   6.00000000e+01
   1.62400000e+01   2.00000000e+00   1.40000000e+01   4.00000000e+00
   5.00000000e+00   7.35000000e+04   1.00000000e+00   1.00000000e+00
   3.50000000e+01   2.86500000e+01   1.00000000e+00   1.00000000e+00
   1.60000000e+01   0.00000000e+00   2.98280000e+04   6.53000000e+01
   5.30000000e+01   0.00000000e+00   3.91561000e+03   0.00000000e+00
   0.00000000e+00   0.0

In [5]:
#Scaling
# X = preprocessing.scale(X)
# test = preprocessing.scale(test)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Random Forest

In [6]:
#Model2 - Random Forest
clf = ensemble.RandomForestClassifier(n_estimators=20)

In [119]:
cross_val_score(clf, X, Y, cv=10)

array([ 0.84670573,  0.84441147,  0.84657138,  0.84343482,  0.84536934,
        0.84497493,  0.84807393,  0.84621164,  0.84564817,  0.84818376])

In [7]:
# clf.fit(X_train, y_train)
clf.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [8]:
forestPred = clf.predict_proba(test)
forestPred = [x[1] for x in forestPred]

In [150]:
roc_auc_score(y_test, forestPred)

ValueError: Found arrays with inconsistent numbers of samples: [106486 354951]

## Logistic

In [9]:
#Model3 - Logistic Regression
clf = LogisticRegression(C=1.0)

In [82]:
cross_val_score(clf, X, Y, cv=10)

array([ 0.78865224,  0.78442237,  0.785831  ,  0.78998178,  0.78545537,
        0.78791578,  0.7918224 ,  0.78808084,  0.7865407 ,  0.78840014])

In [11]:
clf = LogisticRegression(C=1.0)
clf.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
logisticPred = clf.predict_proba(test)
logisticPred = [x[1] for x in logisticPred]

In [121]:
roc_auc_score(y_test, logisticPred)

0.85977445621993398

## GBM

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [17]:
clf = GradientBoostingClassifier(n_estimators=30)

In [90]:
cross_val_score(clf, X, Y, cv=10)

array([ 0.79824957,  0.79634882,  0.79824578,  0.79841482,  0.79708131,
        0.79661176,  0.79753207,  0.7962323 ,  0.79717141,  0.79681454])

In [18]:
# clf.fit(X_train, y_train)
clf.fit(X, Y)

GradientBoostingClassifier(init=None, learning_rate=0.1, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=30,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [19]:
gbmPred = clf.predict_proba(test)
gbmPred = [x[1] for x in gbmPred]

In [129]:
roc_auc_score(y_test, gbmPred)

0.73432072744052479

In [20]:
predictions = [(sum(x)/3) for x in zip(gbmPred, logisticPred, forestPred)]


In [21]:
series = pd.Series(predictions)

In [142]:
roc_auc_score(y_test, predictions)

0.86814966114060399

## Submission file preparation

In [66]:
#Submission predictions
clf = ensemble.RandomForestClassifier(n_estimators=30)
clf.fit(X, Y)
predictions = clf.predict_proba(test)

In [22]:
# series = pd.Series([x[1] for x in predictions])
submission = pd.DataFrame({
     "member_id":testDf['member_id'],
     "loan_status":series,
     })
submission.to_csv('submission4.csv', index=False)

In [50]:
#Model3 - KERAS neural network

# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = Sequential()
model.add(Dense(12, input_dim=X.shape[1], init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10)


# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
